### Detector de matriculas P4 😊

Importamos las librerias necesarias

In [8]:
import cv2
import pytesseract
import re
from ultralytics import YOLO
import easyocr
import csv
import pytesseract
import math
import easyocr

### Modelo usando easyOCR para la deteccion de matriculas

In [ ]:

# Cargamos el modelo YOLO para detección de vehículos y personas
detection_model = YOLO('models/yolov8n.pt')

# Cargamos el modelo YOLO preentrenado para detectar matrículas
license_plate_model = YOLO('models/best.pt')

# Clases de interés (personas y vehículos)
classNames = ["person", "bicycle", "car", "motorbike", "bus", "truck"]

# Inicializamos el lector de EasyOCR
reader = easyocr.Reader(['es'])

# Ruta del video
video_path = 'media/video.mp4'
cap = cv2.VideoCapture(video_path)

# Inicializamos el archivo CSV
csv_file = open("data/detected_objects_easyOCR.csv", mode='w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['frame', 'object_type', 'confidence', 'tracking_id', 'x1', 'y1', 'x2', 'y2', 'license_plate', 'plate_confidence', 'direction'])

# Inicializamos el objeto para grabar el video con los resultados
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('media/output_video_easyOCR.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

frame_count = 0
directions = {"left": 0, "right": 0, "up": 0, "down": 0}  # Contadores de dirección

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break  # Salimos del bucle si se acaba el video

    frame_count += 1
    detection_results = detection_model(img, stream=True)  # Detección de vehículos y personas

    # Procesamos los resultados de detección de YOLO
    for r in detection_results:
        boxes = r.boxes

        for box in boxes:

            # Tomamos las coordenadas del contenedor
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = math.ceil((box.conf[0] * 100)) / 100  # Confianza
            cls = int(box.cls[0])  # Índice de clase

            if cls < len(classNames):
                class_name = classNames[cls]

                # Anonimizamos las personas
                if class_name == "person":

                    # Aplicamos un desenfoque a la región de la persona
                    person_roi = img[y1:y2, x1:x2]
                    blurred_person = cv2.GaussianBlur(person_roi, (51, 51), 0)
                    img[y1:y2, x1:x2] = blurred_person

                    # Detectamos la dirección de la persona
                    direction = "left" if x2 < img.shape[1] // 2 else "right"
                    directions[direction] += 1

                elif class_name in ["car", "motorbike", "bus", "truck"]:
                    direction = "left" if x2 < img.shape[1] // 2 else "right"
                    directions[direction] += 1

                # Seguimiento (si está disponible)
                tracking_id = box.track_id if hasattr(box, 'track_id') else None

                # Dibujamos el rectángulo de la detección
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img, f"{class_name} {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

                # Si es un vehículo, aplicamos el modelo de matrículas
                plate_text = ""
                if class_name in ["car", "motorbike", "bus", "truck"]:

                    img_cropped = img[y1:y2, x1:x2] # Nos quedamos con la zona de region del vehiculo ROI

                    if img_cropped.size > 0:
                        # Detectamos la matrícula con el modelo entrenado para matrículas
                        license_plate_results = license_plate_model(img_cropped, stream=True)

                        # Procesamos el generador directamente
                        for plate in license_plate_results:
                            plate_boxes = plate.boxes
                            for plate_box in plate_boxes:
                                # Extraemos las coordenadas de la matrícula
                                px1, py1, px2, py2 = map(int, plate_box.xyxy[0])
                                img_plate_cropped = img_cropped[py1:py2, px1:px2]
                                
                                # Leemos el texto de la matrícula usando EasyOCR
                                result = reader.readtext(img_plate_cropped, allowlist='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ')
                                
                                # Procesamos el texto detectado
                                if len(result) >= 1:
                                    plate_text = re.sub(r'[^A-Za-z0-9]', '', result[0][1]).strip()  # Limpiamos el texto

                                # Dibujamos el texto de la matrícula en la imagen
                                if plate_text:
                                    cv2.putText(img, plate_text, (x1, y2 + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                                    print("Matrícula detectada:", plate_text)
                                else:
                                    print("No se detectó matrícula.")

                # Escribimos los datos en el archivo CSV
                csv_writer.writerow([frame_count, class_name, confidence, tracking_id, x1, y1, x2, y2, plate_text, confidence, direction])

    # Guardamos el frame procesado en el video de salida
    out.write(img)

    # Mostramos el video con las detecciones
    cv2.imshow('Detector de personas y vehículos', img)

    # Salimos si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberamos los recursos
cap.release()
out.release()
csv_file.close()
cv2.destroyAllWindows()

# Imprimimos el conteo de direcciones
print("Direcciones detectadas:")
for direction, count in directions.items():
    print(f"{direction}: {count}")


Resultado final:

left: 1510,
right: 1175,
up: 0,
down: 0,

Como podemos ver el modelo usando easyOCR para la interpretacion de los caracteres funciona relativamente bien en este caso

### Otro posible modelo usando pytesseract para la deteccion de matriculas

In [ ]:
# Configuramos la ruta de Tesseract
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

# Cargamos el modelo YOLO para detección de vehículos y personas
detection_model = YOLO('models/yolov8n.pt')

# Cargamos el modelo YOLO preentrenado para detectar matrículas
license_plate_model = YOLO('models/modelo.pt')

# Clases de interés (personas y vehículos)
classNames = ["person", "bicycle", "car", "motorbike", "bus", "truck"]

# Ruta del video
video_path = 'media/video.mp4'
cap = cv2.VideoCapture(video_path)

# Inicializamos el archivo CSV a crear
csv_file = open("data/detected_objects_pytesseract.csv", mode='w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['frame', 'object_type', 'confidence', 'tracking_id', 'x1', 'y1', 'x2', 'y2', 'license_plate', 'plate_confidence', 'direction'])

# Inicializamos el objeto para grabar el video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('media/output_video_pytesseract.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

frame_count = 0
directions = {"left": 0, "right": 0, "up": 0, "down": 0}  # Contadores de direccion

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break  # Salimos del bucle si se acaba el video

    frame_count += 1
    detection_results = detection_model(img, stream=True)  # Detección de vehículos y personas

    # Procesamos los resultados de detección de YOLO
    for r in detection_results:
        boxes = r.boxes

        for box in boxes:
            # Tomamos las coordenadas del contenedor
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = math.ceil((box.conf[0] * 100)) / 100  # Confianza
            cls = int(box.cls[0])  # Índice de clase

            if cls < len(classNames):
                class_name = classNames[cls]

                # Anonimizar las personas
                if class_name == "person":
                    # Aplicamos un desenfoque a la región de la persona
                    person_roi = img[y1:y2, x1:x2]
                    blurred_person = cv2.GaussianBlur(person_roi, (51, 51), 0)
                    img[y1:y2, x1:x2] = blurred_person
                    direction = "left" if x2 < img.shape[1] // 2 else "right"
                    directions[direction] += 1

                elif class_name in ["car", "motorbike", "bus", "truck"]:
                    direction = "left" if x2 < img.shape[1] // 2 else "right"
                    directions[direction] += 1

                # Seguimiento (si está disponible)
                tracking_id = box.track_id if hasattr(box, 'track_id') else None

                # Dibujamos el rectángulo de la detección
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img, f"{class_name} {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

                # Si es un vehículo, aplicamos el modelo de matrículas
                plate_text = ""
                if class_name in ["car", "motorbike", "bus", "truck"]:
                    
                    img_cropped = img[y1:y2, x1:x2] # Nos quedamos con los pixeles reconocidos como vehiculo ROI

                    if img_cropped.size > 0:
                        # Aplicamos el modelo para detectar matrículas
                        license_plate_results = license_plate_model(img_cropped, stream=True)

                        # Procesamos el generador directamente
                        for plate in license_plate_results:
                            plate_boxes = plate.boxes
                            for plate_box in plate_boxes:
                                # Extraemos las coordenadas de la matrícula
                                px1, py1, px2, py2 = map(int, plate_box.xyxy[0])
                                img_plate_cropped = img_cropped[py1:py2, px1:px2]
                                
                                # Leemos el texto de la matrícula usando pytesseract
                                text = pytesseract.image_to_string(img_plate_cropped, config='--psm 8 --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
                                plate_text = re.sub(r'[^A-Za-z0-9]', '', text).strip()  # Limpiamos el texto

                                # Dibujamos el texto de la matrícula en la imagen
                                if plate_text:
                                    cv2.putText(img, plate_text, (x1, y2 + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                                    print("Matrícula detectada:", plate_text)
                                else:
                                    print("No se detectó matrícula.")

                # Escribimos los datos en el archivo CSV
                csv_writer.writerow([frame_count, class_name, confidence, tracking_id, x1, y1, x2, y2, plate_text, confidence, direction])

    # Guardamos el frame procesado en el video de salida
    out.write(img)

    # Mostramos el video con las detecciones
    cv2.imshow('Detector de personas y vehículos', img)

    # Salimos si se presiona "q"
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
csv_file.close()
cv2.destroyAllWindows()

print("Direcciones detectadas:")
for direction, count in directions.items():
    print(f"{direction}: {count}")

Resultado final:

left: 1510,
right: 1175,
up: 0,
down: 0,

Como podemos ver este 2 modelo funciona como el anterior aunque falla mas en seguir las matriculas, y predecirlas sin fallar de manera correcta.

Ambos modelos planteamos la estrategia de dividir la deteccion en dos partes primero, dividimos el 1 modelo es el general de yolo para poder detectar personas, y el segundo es un modelo especifico para matriculas, y utilizamos easyOCR en el primero y pyteserrecact en el segundo. Como podemos observar la libreria EasyOCR funciona mucho mejor en este caso.